## Resolución de equilibrio químico del candy

aqui explico la reacción

#### Definición de reacciones de equilibrio y funciones

In [24]:
##-------------------------Ecuaciones de equilibrio------------------------------------

Eqn_1 = ['O','H','OH'] 
coef_1 = [-1,-1,1] 

Eqn_2 = ['O','O2']
coef_2 = [-2,1]

Eqn_3 = ['H','O2','OH','O']
coef_3 = [-1,-1,1,1]

Eqn_4 = ['H','OH','H2O']
coef_4 = [-1,-1,1]

Eqn_5 = ['H2','O','OH','H']
coef_5 = [-1,-1,1,1]

Eqn_6 = ['H2','OH','H2O','H']
coef_6 = [-1,-1,1,1]

Eqn_7 = ['H2O','O','OH']
coef_7 = [-1,-1,2]

##-----------------------------------Librerías--------------------------------------------------------

import matplotlib.pyplot as plt
import sympy as sp
import math
import json
from IPython.display import display, Markdown
import numpy as np
from scipy.optimize import minimize
import os
import pandas as pd

##///////////////////////////////---Cargado de datos y propiedades---///////////////////////////////////////////////

def cargar_datos(ruta):
    with open(ruta, 'r') as archivo:
        datos = json.load(archivo)
    return datos

def obtener_propiedades(datos, nombre_especie, propiedad):
    especie = datos.get(nombre_especie)
    if especie:
        return especie.get(propiedad)
    else:
        return None
   
##/////////////////////////////////---Entalpía, entropía y Cp---//////////////////////////////////////////////////////

def Cp_T(R,T,a_i,e_i):
    C_p = 0
    for index,i in enumerate(a_i):
        C_p += R*(a_i[index]*T**(e_i[index]))
    return C_p

def H_H0(R,T,a_i):
    T_0 = 298.15
    H = -a_i[0]/T + a_i[1]*math.log(T) + a_i[2]*T + a_i[3]*(T**2)/2 + a_i[4]*(T**3)/3 + a_i[5]*(T**4)/4 + a_i[6]*(T**5)/5
    H_0 = -a_i[0]/T_0 + a_i[1]*math.log(T_0) + a_i[2]*T_0 + a_i[3]*(T_0**2)/2 + a_i[4]*(T_0**3)/3 + a_i[5]*(T_0**4)/4 + a_i[6]*(T_0**5)/5
    return R * (H - H_0)/1000 # KJ/Kmol = J/mol

def S(R,T,a_i,b_i):
    S = -a_i[0]*(T**-2) - a_i[1]/T + a_i[2]*math.log(T) + a_i[3]*T + a_i[4]*(T**2)/2 + a_i[5]*(T**3)/3 + a_i[6]*(T**4)/4 + b_i[1]
    return R *(S) # J/mol K
 

##////////////////////////////////////---Kp_T----/////////////////////////////////////////////////////////

def Kp_T(Eqn_1,coef_1,T,ruta_archivos):
    ## Constantes universales
    R = 8.31415 # KJ/ Kmol K = J/mol K

    # Fuentes y datos predeterminados
    j = len(Eqn_1)
    tipo = 'Especies'
    datos = cargar_datos(ruta_archivos.get(tipo))

    ## Obtención de datos de intervalo de temperatura (en este caso 200-1000)
    M_i = [0]*j
    a_i = [0]*j
    b_i = [0]*j
    e_i = [0]*j
    T_int = [0]*j

    for index,i in enumerate(Eqn_1):
        M_i[index] = obtener_propiedades(datos, i, 'MolecWeight')[0]
        a_i[index] = obtener_propiedades(datos, i, 'Intervals')[0]['a']
        b_i[index] = obtener_propiedades(datos, i, 'Intervals')[0]['b']
        e_i[index] = obtener_propiedades(datos, i, 'Intervals')[0]['exp']
        T_int[index] = obtener_propiedades(datos, i, 'Intervals')[0]['T_int']

    ## Obtención entalpía de formación
    hf_i = [0]*j

    for index,i in enumerate(Eqn_1):
        hf_i[index] = (obtener_propiedades(datos, i, 'Enthalpy')[0])

    ## Obtención de la entalpía sensible molar (variación respecto del estandar) de cada componente
    hs_T_i = [0]*j

    for index,a in enumerate(a_i):
        hs_T_i[index] = H_H0(R,T,a)

    ## Obtención de la entalpía molar (variación respecto del estandar) de cada componente (formación + sensible)
    h_T_i = [x + y for x, y in zip(hf_i, hs_T_i)]
    #h_T_i = hs_T_i * 1000

    ## Obtención de la entropía molar para una T de cada componente
    s_T_i = [0]*j
    for index,a in enumerate(a_i):
        b = b_i[index]
        s_T_i[index] = S(R,T,a,b)

    ## Entalpía y entropía molar de la reacción
    h_T_r = 0
    s_T_r = 0

    for index,nu in enumerate(coef_1):
        h_T_r += nu*h_T_i[index]
        s_T_r += nu*s_T_i[index]

    g_T_r = h_T_r - T*s_T_r

    ## Contante de equilibrio para la temperatura T
    K_T = math.exp(-g_T_r/(R*T))

    return K_T 

##////////////////////////////////////---Newton-Raphson----/////////////////////////////////////////////////////////

def NewtonRaphson(hf_,R,coef_p,T_int_3, a_i_1, a_i_2, a_i_ng, x0, tolerance, Especies):

    indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas

    # Convertir x0 a flotante para cálculos numéricos
    x0 = float(x0)
    tramo = abs(2*tolerance)
    tabla = []

    T_0 = 298.15 # K

    cont = 0

    xi = sp.symbols('xi')  # Define xi como símbolo

    fx_ = 0
    dfx_ = 0

    while tramo >= tolerance:
        for index, i in enumerate(Especies):
            if any(ng in i for ng in indent_nogas): #si no es una especie gaseosa
                if  T_int_3[index][1] > x0:
                    #print(f'Temperatura fuera de rango (inferior al intervalo de la especie), elija otra fase de {i}')
                    #break 
                    a_i = a_i_ng[index]  # en parte es eroneo, debería estar lo comentado arriba
                    print(f'diferencia de rango (por debajo) de: {T_int_3[index][1]-x0}K') 
                elif x0 == T_int_3[index][1]:
                    a_i = a_i_ng[index]
                elif T_int_3[index][1] < x0 < T_int_3[index][2]:
                    a_i = a_i_ng[index]
                elif  T_int_3[index][2] < x0:
                    #print(f'Temperatura fuera de rango (inferior al intervalo de la especie), elija otra fase de {i}')
                    #break 
                    a_i = a_i_ng[index]  # en parte es eroneo, debería estar lo comentado arriba
                    print(f'diferencia de rango (por encima) de: {T_int_3[index][1]-x0}K') 
                else:
                    print(T_int_3[index][0],T_int_3[index][1],T_int_3[index][2])
                    print(x0)
                    print(f'Temperatura fuera de rango (valor no encontrado) elija otra fase de {i}')
                    break  
                
                H_0 = -a_i[0]/T_0 + a_i[1]*math.log(T_0) + a_i[2]*T_0 + a_i[3]*(T_0**2)/2 + a_i[4]*(T_0**3)/3 + a_i[5]*(T_0**4)/4 + a_i[6]*(T_0**5)/5
            # Actualización de fx_i y dfx_i usando xi 
                fx_i = -coef_p[index]* R *(-a_i[0]/xi + a_i[1]*sp.log(xi) + a_i[2]*xi + a_i[3]*(xi**2)/2 + a_i[4]*(xi**3)/3 + a_i[5]*(xi**4)/4 + a_i[6]*(xi**5)/5 - H_0)
                dfx_i = -coef_p[index]* R *(a_i[0]/(xi**2) + a_i[1]/xi + a_i[2] + a_i[3]*xi + a_i[4]*(xi**2) + a_i[5]*(xi**3) + a_i[6]*(xi**4))

                fx_ += fx_i
                dfx_ += dfx_i

            else: # si es especie gaseosa
                if  T_int_3[index][0] < x0 < T_int_3[index][1]:
                    a_i = a_i_1[index]
                elif x0 == T_int_3[index][1]:
                    a_i = a_i_1[index]
                elif T_int_3[index][1] < x0 < T_int_3[index][2]:
                    a_i = a_i_2[index]
                else:
                    print(f'Temperatura (T={x0}K) fuera de rango de {i}')
                    break  
                
                H_0 = -a_i[0]/T_0 + a_i[1]*math.log(T_0) + a_i[2]*T_0 + a_i[3]*(T_0**2)/2 + a_i[4]*(T_0**3)/3 + a_i[5]*(T_0**4)/4 + a_i[6]*(T_0**5)/5
            # Actualización de fx_i y dfx_i usando xi 
                fx_i = -coef_p[index]* R *(-a_i[0]/xi + a_i[1]*sp.log(xi) + a_i[2]*xi + a_i[3]*(xi**2)/2 + a_i[4]*(xi**3)/3 + a_i[5]*(xi**4)/4 + a_i[6]*(xi**5)/5 - H_0)
                dfx_i = -coef_p[index]* R *(a_i[0]/(xi**2) + a_i[1]/xi + a_i[2] + a_i[3]*xi + a_i[4]*(xi**2) + a_i[5]*(xi**3) + a_i[6]*(xi**4))

                fx_ += fx_i
                dfx_ += dfx_i

        fx_ = hf_ + fx_  # ahora las sumo, porque un poco más arriba porque fx_i va con un -, es decir, se resta

        # Incremento en el contador
        cont += 1

        # Definiendo funciones lambda para evaluación numérica
        fx = sp.lambdify(xi, fx_, 'numpy')
        dfx = sp.lambdify(xi, dfx_, 'numpy')

        # Resolvemos la iteración
        xnuevo = x0 - fx(x0)/dfx(x0)
        tramo = abs(xnuevo - x0)
        tabla.append([x0, xnuevo, tramo])
        x0_ = x0        # solo lo uso para la leyenda y la gráfica (nada más)
        x0 = xnuevo

        # Rango de valores para x y f(x)
        x_vals = np.linspace(1000, 3000, 100)
        y_vals = fx(x_vals)

        # Definimos el punto de iteración
        x_punto = x0_
        y_punto = fx(x0_)

        plt.figure(figsize=(10, 6))
        plt.scatter(x_punto, y_punto, color='red', label=f'$T_{cont+1}$ será {round(x0,ndigits=2)} K', zorder=3)
        plt.plot(x_vals, y_vals, label=f'$\Delta h_i(T_{cont} = {round(x0_,ndigits=2)}K)$')
        plt.title('Gráfico de $\\Delta\\bar{h}_i(T)$')
        plt.xlabel('$x$')
        plt.ylabel('$f(x)$')
        plt.legend()
        plt.grid(True)
        plt.show()

    return x0, tabla

##/////////////////////////////---TEMPERATURA DE COMBUSTIÓN ADIABÁTICA----/////////////////////////////////////////////

def T_adiab(Reactantes,Productos,coef_r,coef_p,ruta_archivos,R,T_in_NRaphson,tol_NRaphson):

    indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas
    j = len(Reactantes)  # Obtenemos la longitud de las listas de reactantes para inicializar otras listas
    k = len(Productos) # Obtenemos la longitud de las listas de productos para inicializar otras listas

    datos_r = cargar_datos(ruta_archivos.get('Reactantes'))
    datos_p = cargar_datos(ruta_archivos.get('Especies'))

    Eqn = Reactantes + Productos

    coef =  coef_r + [-x for x in coef_p] # positivos en los reactivos, negativos en los productos (xq pasamos restando los prod)

    hf_i_r = [0]*j
    hf_i_p = [0]*k

    for index,i in enumerate(Reactantes):
        hf_i_r[index] = obtener_propiedades(datos_r, i, 'Enthalpy')[0] # J/mol

    for index,i in enumerate(Productos):
        hf_i_p[index] = obtener_propiedades(datos_p, i, 'Enthalpy')[0] # J/mol

    #print(Productos)
    #print(hf_i_p)

    #hf_i = hf_i_r + hf_i_p

    hf_ = 0
    hf_r = 0#
    hf_p = 0#

    #for h,nu in zip(hf_i,coef): 
    #    hf_ += nu*h # J/mol 

    for h,nu in zip(hf_i_r,coef_r): 
        hf_r += nu*h # J/mol 

    for h,nu in zip(hf_i_p,coef_p): 
        hf_p += nu*h # J/mol 

    hf_ = hf_r - hf_p
    #print(hf_, hf_r, hf_p)

    ## Establecemos el polinomio de la entalpía para nuestras especies de los productos
    T_int = [0]*k # almacena los dos intervalos de temperatura con sus 2 temperaturas
    T_int_3 = [0]*k # almacena el límite superior e inferior total y la forontera entre los 2 intervalos

    a_i_ng = [0]*k # amacena las a_i del intervalo de temperaturas para especies NO gaseosas
    b_i_ng = [0]*k

    a_i_1 = [0]*k # amacena las a_i del intervalo de temperaturas 1 para especies gaseosas
    b_i_1 = [0]*k

    a_i_2 = [0]*k
    b_i_2 = [0]*k

    for index,i in enumerate(Productos):
        if any(ng in i for ng in indent_nogas):
            T_int[index] = [0,obtener_propiedades(datos_p, i, 'Intervals')[0]['T_int'][0]]+obtener_propiedades(datos_p, i, 'Intervals')[0]['T_int']
        else:
            T_int[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['T_int']+obtener_propiedades(datos_p, i, 'Intervals')[1]['T_int']
        
        T_int_3[index] = [T_int[index][0],(T_int[index][1]+T_int[index][2])/2,T_int[index][3]]

    for index,i in enumerate(Productos):
        if any(ng in i for ng in indent_nogas): #si no es una especie gaseosa
            a_i_ng[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['a']
            b_i_ng[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['b']
            
        else: # si es especie gaseosa
            a_i_1[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['a']
            b_i_1[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['b']

            a_i_2[index] = obtener_propiedades(datos_p, i, 'Intervals')[1]['a']
            b_i_2[index] = obtener_propiedades(datos_p, i, 'Intervals')[1]['b']

    def NewtonRaphson_Tad(T_int_3, a_i_1, a_i_2, a_i_ng, x0, tolerance, Especies):
    # Convertir x0 a flotante para cálculos numéricos

        x0 = float(x0)
        tramo = abs(2*tolerance)
        tabla = []

        T_0 = 298.15 # K

        cont = 0

        xi = sp.symbols('xi')  # Define xi como símbolo

        fx_ = 0
        dfx_ = 0

        while tramo >= tolerance:
            for index, i in enumerate(Especies):
                if any(ng in i for ng in indent_nogas): #si no es una especie gaseosa
                    if  T_int_3[index][1] > x0:
                        #print(f'Temperatura fuera de rango (inferior al intervalo de la especie), elija otra fase de {i}')
                        #break 
                        a_i = a_i_ng[index]  # en parte es eroneo, debería estar lo comentado arriba
                        print(f'diferencia de rango (por debajo) de: {T_int_3[index][1]-x0}K') 
                    elif x0 == T_int_3[index][1]:
                        a_i = a_i_ng[index]
                    elif T_int_3[index][1] < x0 < T_int_3[index][2]:
                        a_i = a_i_ng[index]
                    elif  T_int_3[index][2] < x0:
                        #print(f'Temperatura fuera de rango (inferior al intervalo de la especie), elija otra fase de {i}')
                        #break 
                        a_i = a_i_ng[index]  # en parte es eroneo, debería estar lo comentado arriba
                        print(f'diferencia de rango (por encima) de: {T_int_3[index][1]-x0}K') 
                    else:
                        print(T_int_3[index][0],T_int_3[index][1],T_int_3[index][2])
                        print(x0)
                        print(f'Temperatura fuera de rango (valor no encontrado) elija otra fase de {i}')
                        break  
                    
                    H_0 = -a_i[0]/T_0 + a_i[1]*math.log(T_0) + a_i[2]*T_0 + a_i[3]*(T_0**2)/2 + a_i[4]*(T_0**3)/3 + a_i[5]*(T_0**4)/4 + a_i[6]*(T_0**5)/5
                # Actualización de fx_i y dfx_i usando xi 
                    fx_i = -coef_p[index]* R *(-a_i[0]/xi + a_i[1]*sp.log(xi) + a_i[2]*xi + a_i[3]*(xi**2)/2 + a_i[4]*(xi**3)/3 + a_i[5]*(xi**4)/4 + a_i[6]*(xi**5)/5 - H_0)
                    dfx_i = -coef_p[index]* R *(a_i[0]/(xi**2) + a_i[1]/xi + a_i[2] + a_i[3]*xi + a_i[4]*(xi**2) + a_i[5]*(xi**3) + a_i[6]*(xi**4))

                    fx_ += fx_i
                    dfx_ += dfx_i

                else: # si es especie gaseosa
                    if  T_int_3[index][0] < x0 < T_int_3[index][1]:
                        a_i = a_i_1[index]
                    elif x0 == T_int_3[index][1]:
                        a_i = a_i_1[index]
                    elif T_int_3[index][1] < x0 < T_int_3[index][2]:
                        a_i = a_i_2[index]
                    else:
                        print(f'Temperatura (T={x0}K) fuera de rango de {i}')
                        break  
                    
                    H_0 = -a_i[0]/T_0 + a_i[1]*math.log(T_0) + a_i[2]*T_0 + a_i[3]*(T_0**2)/2 + a_i[4]*(T_0**3)/3 + a_i[5]*(T_0**4)/4 + a_i[6]*(T_0**5)/5
                # Actualización de fx_i y dfx_i usando xi 
                    fx_i = -coef_p[index]* R *(-a_i[0]/xi + a_i[1]*sp.log(xi) + a_i[2]*xi + a_i[3]*(xi**2)/2 + a_i[4]*(xi**3)/3 + a_i[5]*(xi**4)/4 + a_i[6]*(xi**5)/5 - H_0)
                    dfx_i = -coef_p[index]* R *(a_i[0]/(xi**2) + a_i[1]/xi + a_i[2] + a_i[3]*xi + a_i[4]*(xi**2) + a_i[5]*(xi**3) + a_i[6]*(xi**4))

                    fx_ += fx_i
                    dfx_ += dfx_i

            fx_ = hf_ + fx_  # ahora las sumo, porque un poco más arriba porque fx_i va con un -, es decir, se resta

            # Incremento en el contador
            cont += 1

            # Definiendo funciones lambda para evaluación numérica
            fx = sp.lambdify(xi, fx_, 'numpy')
            dfx = sp.lambdify(xi, dfx_, 'numpy')

            # Resolvemos la iteración
            xnuevo = x0 - fx(x0)/dfx(x0)
            tramo = abs(xnuevo - x0)
            tabla.append([x0, xnuevo, tramo])
            x0_ = x0        # solo lo uso para la leyenda y la gráfica (nada más)
            x0 = xnuevo

            # Rango de valores para x y f(x)
            x_vals = np.linspace(1000, 3000, 100)
            y_vals = fx(x_vals)

            # Definimos el punto de iteración
            x_punto = x0_
            y_punto = fx(x0_)


            #plt.figure(figsize=(10, 6))
            #plt.scatter(x_punto, y_punto, color='red', label=f'$T_{cont+1}$ será {round(x0,ndigits=2)} K', zorder=3)
            #plt.plot(x_vals, y_vals, label=f'$\Delta h_i(T_{cont} = {round(x0_,ndigits=2)}K)$')
            #plt.title('Gráfico de $\\Delta\\bar{h}_i(T)$')
            #plt.xlabel('$x$')
            #plt.ylabel('$f(x)$')
            #plt.legend()
            #plt.grid(True)
            #plt.show()


        return x0 #, tabla   # Temperatura final , Tabla de valores de convergencia
    
    x0 = T_in_NRaphson
    tolerance = tol_NRaphson

    return NewtonRaphson_Tad(T_int_3, a_i_1, a_i_2, a_i_ng, x0, tolerance, Productos)


##/////////////////////////////////---Extracción de datos de combustión---//////////////////////////////////////////////////////


def Datos_ProductosComb(ruta_archivos,Productos,Tc,R):
    '''
    Extrae Mm, e_i, a_i y b_i de cada uno de los componentes de los productos
    para después poder realizar operaciones con ellos de manera sencilla
    '''

    indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas
    
    tipo = 'Especies'
    
    datos_p = cargar_datos(ruta_archivos.get(tipo))
    numero_prod=len(Productos)

    Mm = [0]*numero_prod
    T_int = [0]*numero_prod
    T_int_3 = [0]*numero_prod
    e_i = [0]*numero_prod
    a_i = [0]*numero_prod
    b_i = [0]*numero_prod
    a_i_ng = [0]*numero_prod
    b_i_ng = [0]*numero_prod
    a_i_1 = [0]*numero_prod
    a_i_2 = [0]*numero_prod
    b_i_1 = [0]*numero_prod
    b_i_2 = [0]*numero_prod
    Cp_i = [0]*numero_prod
    Hs_i = [0]*numero_prod
    S_i = [0]*numero_prod

    ## Obtenemos sus masas moleculares y sus intervalos
    for index,i in enumerate(Productos):
        Mm[index] = obtener_propiedades(datos_p, i, 'MolecWeight')[0]
        e_i[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['exp']
        if any(ng in i for ng in indent_nogas):
            T_int[index] = [0,obtener_propiedades(datos_p, i, 'Intervals')[0]['T_int'][0]]+obtener_propiedades(datos_p, i, 'Intervals')[0]['T_int']
            a_i_ng[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['a']
            b_i_ng[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['b']        
        else:
            T_int[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['T_int']+obtener_propiedades(datos_p, i, 'Intervals')[1]['T_int']
            a_i_1[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['a']
            b_i_1[index] = obtener_propiedades(datos_p, i, 'Intervals')[0]['b']

            a_i_2[index] = obtener_propiedades(datos_p, i, 'Intervals')[1]['a']
            b_i_2[index] = obtener_propiedades(datos_p, i, 'Intervals')[1]['b']

        T_int_3[index] = [T_int[index][0],(T_int[index][1]+T_int[index][2])/2,T_int[index][3]]

    ## Asignamos los intervalos en función de la temperatura de combustión obtenida
    for index, i in enumerate(Productos):
        if any(ng in i for ng in indent_nogas): #si no es una especie gaseosa
            if  T_int_3[index][1] > Tc:
                print(f'Temperatura fuera de rango (inferior al intervalo de la especie), elija otra fase de {i}')
                break 
                #a_i = a_i_ng[index]  # en parte es eroneo, debería estar lo comentado arriba
                #b_i = b_i_ng[index]
                #print(f'diferencia de rango (por debajo) de: {T_int_3[index][1]-Tc}K') 
            elif Tc == T_int_3[index][1]:
                a_i[index] = a_i_ng[index]
                b_i[index] = b_i_ng[index]
            elif T_int_3[index][1] < Tc < T_int_3[index][2]:
                a_i[index] = a_i_ng[index]
                b_i[index] = b_i_ng[index]
            elif  T_int_3[index][2] < Tc:
                print(f'Temperatura fuera de rango (inferior al intervalo de la especie), elija otra fase de {i}')
                break 
                #a_i = a_i_ng[index]  # en parte es eroneo, debería estar lo comentado arriba
                #b_i = b_i_ng[index]
                #print(f'diferencia de rango (por encima) de: {T_int_3[index][1]-Tc}K') 
            else:
                print(T_int_3[index][0],T_int_3[index][1],T_int_3[index][2])
                print(Tc)
                print(f'Temperatura fuera de rango (valor no encontrado) elija otra fase de {i}')
                break  

        else: # si es especie gaseosa
            if  T_int_3[index][0] < Tc < T_int_3[index][1]:
                a_i[index] = a_i_1[index]
                b_i[index] = b_i_1[index]
            elif Tc == T_int_3[index][1]:
                a_i[index] = a_i_1[index]
                b_i[index] = b_i_1[index]
            elif T_int_3[index][1] < Tc < T_int_3[index][2]:
                a_i[index] = a_i_2[index]
                b_i[index] = b_i_2[index]
            else:
                print(f'Temperatura (T={Tc}K) fuera de rango de {i}')
                break 

    ## Calculamos Cp, Hs y S para la temperatura dada
        Cp_i[index] = Cp_T(R,Tc,a_i[index],e_i[index])
        Hs_i[index] = H_H0(R,Tc,a_i[index])
        S_i[index] = S(R,Tc,a_i[index],b_i[index])

    return Mm,e_i,a_i,b_i,Cp_i,Hs_i,S_i

<>:227: SyntaxWarning: invalid escape sequence '\D'
<>:227: SyntaxWarning: invalid escape sequence '\D'
C:\Users\german.perez\AppData\Local\Temp\ipykernel_57508\2200325066.py:227: SyntaxWarning: invalid escape sequence '\D'
  plt.plot(x_vals, y_vals, label=f'$\Delta h_i(T_{cont} = {round(x0_,ndigits=2)}K)$')


In [25]:
#/////////////////////////--RUTA DE ARCHIVOS DE BASES DE DATOS--//////////////////////////////////////////
ruta_archivos = {
    'Reactantes' : 'C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/code/Bases de datos/Reactants.json',
    'Especies' : 'C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/code/Bases de datos/Species.json',
    'Propulsantes' : 'C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/code/Bases de datos/Propellants.json'
}

#/////////////////////////--RUTA DE ARCHIVOS DE PRESION Y TIEMPO--//////////////////////////////////////////

# Definir la ruta del archivo
file_path = r"C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/Fotos/6 - Resultados/Resultados Equilibrio Químico/presion y tiempo corto.txt"

# Leer los datos asumiendo separador de espacios y punto decimal
df = pd.read_csv(file_path, sep='\s+', decimal='.', header=None)

# Definir los nombres de las columnas
df.columns = ['Tiempo', 'Presión']

#////////////////////////--ARREGLOS PARA EL GUARDADO--//////////////////////////////////////////////////////////////

# Definir el directorio donde se guardará el archivo
output_dir = "C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/Fotos/6 - Resultados" 

# Crear el directorio si no existe
os.makedirs(output_dir, exist_ok=True)

# Ruta del archivo de salida con el valor del tiempo en el nombre
output_file = os.path.join(output_dir, f"resultados_equilibrio.txt")

output_text="""Tiempo Presión Temperatura Cp Mm Rg Gamma Vsound N_KOH N_K2CO3 N_K2O N_CO N_CO2 N_H2O N_H2 N_H N_OH N_O N_O2 N_N2"""

# Guardar en un archivo de texto
with open(output_file, "w") as f:
    f.write(output_text)

## Constantes universales
R = 8.31415 # KJ/ Kmol K = J/mol K

## Temperaturas y presiones estandar
T_0 = 298.15 # K
p0 = 1.0  # Presión estándar

## Definición de mezcla de reactivos
N_KNO3 = 1
N_C6H14O6 = 0.2988

## Nombre de las especies de la reacción
Reactantes = ['KNO3(s)','Sorbitol']
coef_r = [N_KNO3,N_C6H14O6]
Productos = ['KOH(L)','K2CO3(L)','K2O','CO','CO2','H2O','H2','H','OH','O','O2']						

coef_p = [0.000360065792818802, 0.493640832027809, 0.00272566861913182,
                0.87866000126, 0.40143172266, 1.63439159539, 0.45389515045,
                0.00000846023, 0.00012044938, 0.00000013817, 0.0000000001]
N_N2 = [0.5] # fracción molar de N2
N_m = [4.3619322173912005] # suma de los coef, estequeométricos


#////////////////////////--VALORES INICIALES--//////////////////////////////////////////////////////////////

initial_guess = coef_p+N_m ## Composiciones de N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_m
T_inicial_iteracion = 1559 # K : Temperatura única en la que se analiza Kp
T_in_NRaphson = 1550 # K : Temperatura única en la que se analiza en el NRaphson
x_tot_0 = 0.5 ## Valor inicial de la suma de fracciones molares

#////////////////////////--TOLERANCIAS DE CONVERGENCIA--//////////////////////////////////////////////////////////

tol_x = 0.01 # tolerancia de convergencia para la fracción molar
tol_Tad = 0.001 # tolerancia de convergencia para la T_adiabática
tol_NRaphson = 10 # Tolerancia de convergencia del NRaphson

#////////////////////////--ARREGLOS PARA LA TEMP. ADIAB.--//////////////////////////////////////////////////////////

indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas

################################################################################################################
################################################################################################################
################################################################################################################

#////////////////////////--INICIO DE LAS ITERACIONES.--/////////////////////////////////////////////////////////////

# Iterar sobre cada fila y mostrar los valores de Tiempo y Presión
for index, row in df.iterrows():

    #///////////////--DEFINICIÓN DE CONDICIONES Y REACCIÓN QUÍMICA--//////////////////////////////////////////

    ## Paso de tiempo
    tiempo = row['Tiempo']# En segundos

    ## Presión de cámara
    p = row['Presión'] # Presión de cámara

    #///////////////--INICIO DEL CÁLCULO--//////////////////////////////////////////

    T = T_inicial_iteracion # Antes de entrar al bucle inicializo la temperatura
    T_nueva = T_inicial_iteracion + 100
    contador_iter = 0

    while abs(T-T_nueva) >= tol_Tad:

        contador_iter += 1
        print(f'En la iteración {contador_iter} obtenemos:')

        T = T_nueva # temperatura obtenida en el anterior bucle y la que se usa para la concentración de equilibrio

        ## Constantes de equilibrio
        K_p_H_O_OH = Kp_T(Eqn_1,coef_1,T,ruta_archivos)
        K_p_O_O2 = Kp_T(Eqn_2,coef_2,T,ruta_archivos)
        K_p_H_O2_OH_O = Kp_T(Eqn_3,coef_3,T,ruta_archivos)
        K_p_H_OH_H2O = Kp_T(Eqn_4,coef_4,T,ruta_archivos)
        K_p_H2_O_OH_H = Kp_T(Eqn_5,coef_5,T,ruta_archivos)
        K_p_H2_OH_H2O_H = Kp_T(Eqn_6,coef_6,T,ruta_archivos)
        K_p_H2O_O_OH = Kp_T(Eqn_7,coef_7,T,ruta_archivos)


        # Define the function to compute the residuals as the sum of squares of each equation
        def objective(vars):
            N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_m = vars
            
            eq1 = 2 * N_K2CO3 + 2 * N_K2O + N_KOH - 1*N_KNO3
            eq2 = N_CO + N_CO2 + N_K2CO3 - 6*N_C6H14O6
            eq3 = N_H + 2 * N_H2 + 2 * N_H2O + N_KOH + N_OH - 14*N_C6H14O6
            eq4 = N_CO + 2 * N_CO2 + N_H2O + 3 * N_K2CO3 + N_K2O + N_KOH + N_O + 2 * N_O2 + N_OH - 3*N_KNO3 - 6*N_C6H14O6

            eq5 = N_CO + N_CO2 + N_H + N_H2 + N_H2O + N_K2CO3 + N_K2O + N_KOH + N_O + N_O2 + N_OH - N_m + 0.5

            eq6 = (p0/p) * N_OH * N_m  - K_p_H_O_OH* (N_H * N_O)
            eq7 = (p0/p) * N_O2 * N_m  - K_p_O_O2*N_O**2
            eq8 = N_O * N_OH  - K_p_H_O2_OH_O*(N_H * N_O2)
            eq9 = (p0/p) * N_H2O * N_m - K_p_H_OH_H2O*(N_H * N_OH)
            eq10 = N_H * N_OH  - K_p_H2_O_OH_H*(N_H2 * N_O)
            eq11 = N_H * N_H2O - K_p_H2_OH_H2O_H*(N_H2 * N_OH)
            eq12 = N_OH**2 - K_p_H2O_O_OH*(N_H2O * N_O)

            # Return the sum of squares of the residuals
            return (eq1**2 + eq2**2 + eq3**2 + eq4**2 + eq5**2 +
                    eq6**2 + eq7**2 + eq8**2 + eq9**2 + eq10**2 + eq11**2 + eq12**2)

        x_tot = x_tot_0

        while abs(x_tot-1) >= tol_x:
            #print(x_tot-1,tolerancia)

            # Define bounds for each variable to be greater than zero
            bounds = [(1e-10, None)] * 12  # Use a small number close to zero to avoid division by zero in equations

            # Use 'L-BFGS-B' because it supports bounds
            solution_1 = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')

            #print("Solución:")
            #print(solution_1.x)  # Shows the optimized values of the variables

            initial_guess = solution_1.x.tolist()

            x_i = [0]*len(solution_1.x)
            x_tot = 0.5/solution_1.x[11] # aqui tengo en cuenta ya el nitrógeno
            for index,i in enumerate(solution_1.x):
                x_i[index] = solution_1.x[index]/solution_1.x[11]
                x_tot += x_i[index]
                if index == 10:
                    break

            print(f'Suma de fracciones molares: {x_tot}')

        
        print(f'Temperatura de combustión inicial: {T}K')
        print("Los coeficientes:  N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_N2")
        print(solution_1.x.tolist()[:11]+N_N2)  # Shows the optimized values of the variables
        print(f'Suma de coeficientes (N_m): {solution_1.x.tolist()[11]}')


        Productos_tot = Productos + ['N2']

        coef_p_tot = solution_1.x.tolist()[:11]+N_N2

        ## Ahora calculamos la nueva temperatura de combustión adiabática para la siguiente iteración
        T_nueva = T_adiab(Reactantes,Productos_tot,coef_r,coef_p_tot,ruta_archivos,R,T,tol_NRaphson)

    Coef = solution_1.x.tolist()[:11]+N_N2
    N_m=solution_1.x.tolist()[11]

    Mm_i = Datos_ProductosComb(ruta_archivos,Productos,T_nueva,R)[0]  # Mm,e_i,a_i,b_i,Cp_i,Hs_i,S_i
    Cp_i = Datos_ProductosComb(ruta_archivos,Productos,T_nueva,R)[4]  # Mm,e_i,a_i,b_i,Cp_i,Hs_i,S_i

    ## Cálculo de Cp_mezcla, Mm_mezcla, Gamma_mezcla, Vsonido_mezcla

    indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas
    C_ng, Cp_gas,Cp_mix,Mm_ng,Mm_gas,Mm_mix,N_ng,N_gas = 0,0,0,0,0,0,0,0

    for index, i in enumerate(Productos):
        if any(ng in i for ng in indent_nogas): #si no es una especie gaseosa
            C_ng += (Coef[index])*Cp_i[index]
            Mm_ng += (Coef[index])*Mm_i[index]
            N_ng += Coef[index]
        else:
            Cp_gas += (Coef[index])*Cp_i[index]
            Mm_gas += (Coef[index])*Mm_i[index]
            N_gas += Coef[index]

        Cp_mix += C_ng + Cp_gas
        Mm_mix += Mm_ng + Mm_gas

    C_ng = C_ng/N_ng
    Cp_gas = Cp_gas/N_m #Cp_gas/N_gas
    Cp_mix = Cp_mix/N_m
    Mm_ng = Mm_ng/N_ng
    Mm_gas = Mm_gas/N_gas
    Mm_mix = Mm_mix/N_m

    Gamma_gas = Cp_gas / (Cp_gas-R)
    Gamma_mix = Cp_mix / (Cp_mix-R)

    Vsound_gas = np.sqrt(Gamma_gas*(R/(Mm_gas/1000))*T_nueva)
    Vsound_mix = np.sqrt(Gamma_mix*(R/(Mm_mix/1000))*T_nueva)

    # Datos a escribir en el archivo (El 11 es el numero de moles ojo)
    output_text = f"""
 {tiempo} {round(p, ndigits=2)} {round(T_nueva, ndigits=2)} {round(Cp_gas, ndigits=4)} {round(Mm_gas, ndigits=4)} {round(R/(Mm_gas/1000), ndigits=4)} {round(Gamma_gas, ndigits=4)} {round(Vsound_gas, ndigits=2)} {solution_1.x.tolist()[0]} {solution_1.x.tolist()[1]} {solution_1.x.tolist()[2]} {solution_1.x.tolist()[3]} {solution_1.x.tolist()[4]} {solution_1.x.tolist()[5]} {solution_1.x.tolist()[6]} {solution_1.x.tolist()[7]} {solution_1.x.tolist()[8]} {solution_1.x.tolist()[9]} {solution_1.x.tolist()[10]} {N_N2[0]}"""
    
    print(f"Archivo generado exitosamente en: {output_text}")
    
    # Guardar en un archivo de texto
    with open(output_file, "a") as f:
        f.write(output_text)

# Mostrar la ruta del archivo generado
print(f"Archivo generado exitosamente en: {output_file}")

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
C:\Users\german.perez\AppData\Local\Temp\ipykernel_57508\1803209644.py:14: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file_path, sep='\s+', decimal='.', header=None)


En la iteración 1 obtenemos:
Suma de fracciones molares: 1.0007756578582525
Temperatura de combustión inicial: 1659K
Los coeficientes:  N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_N2
[0.0003600657940146247, 0.4936408320317225, 0.002725668620744906, 0.8786600012642589, 0.40143172266060767, 1.6343915953015455, 0.45389515045442885, 8.072034602871264e-05, 0.00012968930053737897, 1.3822885263916453e-07, 1e-10, 0.5]
Suma de coeficientes (N_m): 4.3619322171013835
En la iteración 2 obtenemos:
Suma de fracciones molares: 1.0005345370203227
Temperatura de combustión inicial: 1559.514975166331K
Los coeficientes:  N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_N2
[0.0003600525048516974, 0.49364079102869973, 0.0027256516855024555, 0.878659957546894, 0.40143171558303997, 1.6329120808934636, 0.45389510393865157, 1.4736548304584455e-05, 6.893521393749401e-05, 1.3280542798635456e-07, 1e-10, 0.5]
Suma de coeficientes (N_m): 4.361377839933713
En la i

In [20]:
#/////////////////////////--RUTA DE ARCHIVOS DE BASES DE DATOS--//////////////////////////////////////////
ruta_archivos = {
    'Reactantes' : 'C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/code/Bases de datos/Reactants.json',
    'Especies' : 'C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/code/Bases de datos/Species.json',
    'Propulsantes' : 'C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/code/Bases de datos/Propellants.json'
}

#/////////////////////////--RUTA DE ARCHIVOS DE PRESION Y TIEMPO--//////////////////////////////////////////

# Definir la ruta del archivo
file_path = r"C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/Fotos/6 - Resultados/Resultados Equilibrio Químico/presion y tiempo.txt"

# Leer los datos asumiendo separador de espacios y punto decimal
df = pd.read_csv(file_path, sep='\s+', decimal='.', header=None)

# Definir los nombres de las columnas
df.columns = ['Tiempo', 'Presión']

#////////////////////////--ARREGLOS PARA EL GUARDADO--//////////////////////////////////////////////////////////////

# Definir el directorio donde se guardará el archivo
output_dir = "C:/Users/german.perez/OneDrive - Destinus NL BV/Escritorio/Personal/TFG/Fotos/6 - Resultados"  # Cambia esto a tu directorio deseado

# Crear el directorio si no existe
os.makedirs(output_dir, exist_ok=True)

# Ruta del archivo de salida con el valor del tiempo en el nombre
output_file = os.path.join(output_dir, f"resultados_equilibrio.txt")

output_text="""Tiempo Presión Temperatura Cp Mm Rg Gamma Vsound N_KOH N_K2CO3 N_K2O N_CO N_CO2 N_H2O N_H2 N_H N_OH N_O N_O2 N_N2"""

# Guardar en un archivo de texto
with open(output_file, "w") as f:
    f.write(output_text)

# Iterar sobre cada fila y mostrar los valores de Tiempo y Presión
for index, row in df.iterrows():

    #///////////////--DEFINICIÓN DE CONDICIONES Y REACCIÓN QUÍMICA--//////////////////////////////////////////

    ## Paso de tiempo
    tiempo = row['Tiempo']# En segundos

    ## Constantes universales
    R = 8.31415 # KJ/ Kmol K = J/mol K

    ## Temperaturas y presiones
    T_0 = 298.15 # K
    p = row['Presión'] # Presión de cámara
    p0 = 1.0  # Presión estándar

    ## Definición de mezcla de reactivos
    N_KNO3 = 1
    N_C6H14O6 = 0.2988

    ## Nombre de las especies de la reacción
    Reactantes = ['KNO3(s)','Sorbitol']
    coef_r = [N_KNO3,N_C6H14O6]
    Productos = ['KOH(L)','K2CO3(L)','K2O','CO','CO2','H2O','H2','H','OH','O','O2']						

    coef_p = [0.000360065792818802, 0.493640832027809, 0.00272566861913182,
                    0.87866000126, 0.40143172266, 1.63439159539, 0.45389515045,
                    0.00000846023, 0.00012044938, 0.00000013817, 0.0000000001]
    N_N2 = [0.5] # fracción molar de N2
    N_m = [4.3619322173912005] # suma de los coef, estequeométricos


    #////////////////////////--VALORES INICIALES--//////////////////////////////////////////////////////////////

    initial_guess = coef_p+N_m ## Composiciones de N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_m
    T_inicial_iteracion = 1559 # K : Temperatura única en la que se analiza Kp
    T_in_NRaphson = 1550 # K : Temperatura única en la que se analiza en el NRaphson
    x_tot_0 = 0.5 ## Valor inicial de la suma de fracciones molares

    #////////////////////////--TOLERANCIAS DE CONVERGENCIA--//////////////////////////////////////////////////////////

    tol_x = 0.01 # tolerancia de convergencia para la fracción molar
    tol_Tad = 0.001 # tolerancia de convergencia para la T_adiabática
    tol_NRaphson = 10 # Tolerancia de convergencia del NRaphson

    #////////////////////////--ARREGLOS PARA LA TEMP. ADIAB.--//////////////////////////////////////////////////////////

    indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas
        
    ################################################################################################################
    ################################################################################################################
    ################################################################################################################

    T = T_inicial_iteracion # Antes de entrar al bucle inicializo la temperatura
    T_nueva = T_inicial_iteracion + 100
    contador_iter = 0

    while abs(T-T_nueva) >= tol_Tad:

        contador_iter += 1
        print(f'En la iteración {contador_iter} obtenemos:')

        T = T_nueva # temperatura obtenida en el anterior bucle y la que se usa para la concentración de equilibrio

        ## Constantes de equilibrio
        K_p_H_O_OH = Kp_T(Eqn_1,coef_1,T,ruta_archivos)
        K_p_O_O2 = Kp_T(Eqn_2,coef_2,T,ruta_archivos)
        K_p_H_O2_OH_O = Kp_T(Eqn_3,coef_3,T,ruta_archivos)
        K_p_H_OH_H2O = Kp_T(Eqn_4,coef_4,T,ruta_archivos)
        K_p_H2_O_OH_H = Kp_T(Eqn_5,coef_5,T,ruta_archivos)
        K_p_H2_OH_H2O_H = Kp_T(Eqn_6,coef_6,T,ruta_archivos)
        K_p_H2O_O_OH = Kp_T(Eqn_7,coef_7,T,ruta_archivos)


        # Define the function to compute the residuals as the sum of squares of each equation
        def objective(vars):
            N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_m = vars
            
            eq1 = 2 * N_K2CO3 + 2 * N_K2O + N_KOH - 1*N_KNO3
            eq2 = N_CO + N_CO2 + N_K2CO3 - 6*N_C6H14O6
            eq3 = N_H + 2 * N_H2 + 2 * N_H2O + N_KOH + N_OH - 14*N_C6H14O6
            eq4 = N_CO + 2 * N_CO2 + N_H2O + 3 * N_K2CO3 + N_K2O + N_KOH + N_O + 2 * N_O2 + N_OH - 3*N_KNO3 - 6*N_C6H14O6

            eq5 = N_CO + N_CO2 + N_H + N_H2 + N_H2O + N_K2CO3 + N_K2O + N_KOH + N_O + N_O2 + N_OH - N_m + 0.5

            eq6 = (p0/p) * N_OH * N_m  - K_p_H_O_OH* (N_H * N_O)
            eq7 = (p0/p) * N_O2 * N_m  - K_p_O_O2*N_O**2
            eq8 = N_O * N_OH  - K_p_H_O2_OH_O*(N_H * N_O2)
            eq9 = (p0/p) * N_H2O * N_m - K_p_H_OH_H2O*(N_H * N_OH)
            eq10 = N_H * N_OH  - K_p_H2_O_OH_H*(N_H2 * N_O)
            eq11 = N_H * N_H2O - K_p_H2_OH_H2O_H*(N_H2 * N_OH)
            eq12 = N_OH**2 - K_p_H2O_O_OH*(N_H2O * N_O)

            # Return the sum of squares of the residuals
            return (eq1**2 + eq2**2 + eq3**2 + eq4**2 + eq5**2 +
                    eq6**2 + eq7**2 + eq8**2 + eq9**2 + eq10**2 + eq11**2 + eq12**2)

        x_tot = x_tot_0

        while abs(x_tot-1) >= tol_x:
            #print(x_tot-1,tolerancia)

            # Define bounds for each variable to be greater than zero
            bounds = [(1e-10, None)] * 12  # Use a small number close to zero to avoid division by zero in equations

            # Use 'L-BFGS-B' because it supports bounds
            solution_1 = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')

            #print("Solución:")
            #print(solution_1.x)  # Shows the optimized values of the variables

            initial_guess = solution_1.x.tolist()

            x_i = [0]*len(solution_1.x)
            x_tot = 0.5/solution_1.x[11] # aqui tengo en cuenta ya el nitrógeno
            for index,i in enumerate(solution_1.x):
                x_i[index] = solution_1.x[index]/solution_1.x[11]
                x_tot += x_i[index]
                if index == 10:
                    break

            print(f'Suma de fracciones molares: {x_tot}')

        
        print(f'Temperatura de combustión inicial: {T}K')
        print("Los coeficientes:  N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_N2")
        print(solution_1.x.tolist()[:11]+N_N2)  # Shows the optimized values of the variables
        print(f'Suma de coeficientes (N_m): {solution_1.x.tolist()[11]}')


        Productos_tot = Productos + ['N2']

        coef_p_tot = solution_1.x.tolist()[:11]+N_N2

        ## Ahora calculamos la nueva temperatura de combustión adiabática para la siguiente iteración
        T_nueva = T_adiab(Reactantes,Productos_tot,coef_r,coef_p_tot,ruta_archivos,R,T,tol_NRaphson)

    Coef = solution_1.x.tolist()[:11]+N_N2
    N_m=solution_1.x.tolist()[11]

    Mm_i = Datos_ProductosComb(ruta_archivos,Productos,T_nueva,R)[0]  # Mm,e_i,a_i,b_i,Cp_i,Hs_i,S_i
    Cp_i = Datos_ProductosComb(ruta_archivos,Productos,T_nueva,R)[4]  # Mm,e_i,a_i,b_i,Cp_i,Hs_i,S_i

    ## Cálculo de Cp_mezcla, Mm_mezcla, Gamma_mezcla, Vsonido_mezcla

    indent_nogas = ['(L)','(s)','(a)','(b)'] # identificadores de no gas
    C_ng, Cp_gas,Cp_mix,Mm_ng,Mm_gas,Mm_mix,N_ng,N_gas = 0,0,0,0,0,0,0,0

    for index, i in enumerate(Productos):
        if any(ng in i for ng in indent_nogas): #si no es una especie gaseosa
            C_ng += (Coef[index])*Cp_i[index]
            Mm_ng += (Coef[index])*Mm_i[index]
            N_ng += Coef[index]
        else:
            Cp_gas += (Coef[index])*Cp_i[index]
            Mm_gas += (Coef[index])*Mm_i[index]
            N_gas += Coef[index]

        Cp_mix += C_ng + Cp_gas
        Mm_mix += Mm_ng + Mm_gas

    C_ng = C_ng/N_ng
    Cp_gas = Cp_gas/N_m #Cp_gas/N_gas
    Cp_mix = Cp_mix/N_m
    Mm_ng = Mm_ng/N_ng
    Mm_gas = Mm_gas/N_gas
    Mm_mix = Mm_mix/N_m

    Gamma_gas = Cp_gas / (Cp_gas-R)
    Gamma_mix = Cp_mix / (Cp_mix-R)

    Vsound_gas = np.sqrt(Gamma_gas*(R/(Mm_gas/1000))*T_nueva)
    Vsound_mix = np.sqrt(Gamma_mix*(R/(Mm_mix/1000))*T_nueva)

    # Datos a escribir en el archivo (El 11 es el numero de moles ojo)
    output_text = f"""
 {tiempo} {round(p, ndigits=2)} {round(T_nueva, ndigits=2)} {round(Cp_gas, ndigits=4)} {round(Mm_gas, ndigits=4)} {round(R/(Mm_gas/1000), ndigits=4)} {round(Gamma_gas, ndigits=4)} {round(Vsound_gas, ndigits=2)} {solution_1.x.tolist()[0]} {solution_1.x.tolist()[1]} {solution_1.x.tolist()[2]} {solution_1.x.tolist()[3]} {solution_1.x.tolist()[4]} {solution_1.x.tolist()[5]} {solution_1.x.tolist()[6]} {solution_1.x.tolist()[7]} {solution_1.x.tolist()[8]} {solution_1.x.tolist()[9]} {solution_1.x.tolist()[10]} {N_N2[0]}"""
    
    print(f"Archivo generado exitosamente en: {output_text}")
    
    # Guardar en un archivo de texto
    with open(output_file, "a") as f:
        f.write(output_text)

# Mostrar la ruta del archivo generado
print(f"Archivo generado exitosamente en: {output_file}")

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
C:\Users\german.perez\AppData\Local\Temp\ipykernel_57508\2767006338.py:14: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file_path, sep='\s+', decimal='.', header=None)


En la iteración 1 obtenemos:
Suma de fracciones molares: 1.0007756578582525
Temperatura de combustión inicial: 1659K
Los coeficientes:  N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_N2
[0.0003600657940146247, 0.4936408320317225, 0.002725668620744906, 0.8786600012642589, 0.40143172266060767, 1.6343915953015455, 0.45389515045442885, 8.072034602871264e-05, 0.00012968930053737897, 1.3822885263916453e-07, 1e-10, 0.5]
Suma de coeficientes (N_m): 4.3619322171013835
En la iteración 2 obtenemos:
Suma de fracciones molares: 1.0005345370203227
Temperatura de combustión inicial: 1559.514975166331K
Los coeficientes:  N_KOH, N_K2CO3, N_K2O, N_CO, N_CO2, N_H2O, N_H2, N_H, N_OH, N_O, N_O2, N_N2
[0.0003600525048516974, 0.49364079102869973, 0.0027256516855024555, 0.878659957546894, 0.40143171558303997, 1.6329120808934636, 0.45389510393865157, 1.4736548304584455e-05, 6.893521393749401e-05, 1.3280542798635456e-07, 1e-10, 0.5]
Suma de coeficientes (N_m): 4.361377839933713
En la i